### 구축환경
- hadoop system 위에 spark 를 포함시켜 놓고 pyspark 하고 cluster로 연동해놓은 상태
    - hadoop spark infra 를 이용해서 base programing 작성 시 사용
    - spark context 와 spark session 은 자동으로 객체 생성되어서 제공됨
    - sc, spark 를 생성하지 않고 사용
- piplline 국축할 때는 자동화 연결시에는 cluster를 pyspark로 연결하지 않고 pyspark 독립적으로 실행
    - spark context 와 spark session 은 직접 만들어서 사용

In [1]:
stocks = [
    ('Google', 'GOOGL', 'USA', 2984, 'USD'), 
    ('Netflix', 'NFLX', 'USA', 645, 'USD'),
    ('Amazon', 'AMZN', 'USA', 3518, 'USD'),
    ('Tesla', 'TSLA', 'USA', 1222, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 483, 'HKD'),
    ('Toyota', '7203', 'Japan', 2006, 'JPY'),
    ('Samsung', '005930', 'Korea', 70600, 'KRW'),
    ('Kakao', '035720', 'Korea', 125000, 'KRW'),
]
stockSchema = ["name", "ticker", "country", "price", "currency"]
df = spark.createDataFrame(data=stocks, schema=stockSchema) # 지연연산

### spark dataframe 생성 방법
1. Row 객체 사용
2. list 안에 df 의 행 데이터를 원소로 구성
3. 기존 hdfs(여러 종류 dataset 사용가능)의 data를 load 해서 생성: spark.read.csv, spark.read.json 모듈 사용
- spark dataframe 은 내부에 rdd 자료 구조를 포함하고 있다.
    - dataframe 의 schema 가 필요한 이유

In [2]:
df.show() # action 연산 : 생성 계획한 df를 연산해서 일부를 추출해서 보야줄 것

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



 1. df에서 name, ticker를 추출하시오
- df명.select('컬럼명',....)
- select(col('컬럼명',....))
- select(df이름.'컬럼명',...)
- select(집계함수,...)

In [4]:
df.select('name', 'ticker') # 지연연산
df.select('name', 'ticker').show() # action 연산 연결

+-------+------+
|   name|ticker|
+-------+------+
| Google| GOOGL|
|Netflix|  NFLX|
| Amazon|  AMZN|
|  Tesla|  TSLA|
|Tencent|  0700|
| Toyota|  7203|
|Samsung|005930|
|  Kakao|035720|
+-------+------+



2. df를 stocks 라는 view로 생성하시오
- spark 는 database 기술에서 사용하는 db 처리 언어인 spl 을 거의 유사하게 사용할 수 있음
- spark sql 을 사용하기 위해서는 dataframe 을 spark view 로 변환, 생성해서 사용해야 함
    - spark view 는 db 에서 테이블과 동일하게 사용할 수 있음
- view 생성 매서드
    - df.createOrReplaceTempView(view 이름)

In [10]:
df.createOrReplaceTempView('stocks') # stocks view 사용가능 상태

3. view stocks를 사용하여 stocks view의 모든 컬럼을 추출하시오
- sql: select ~ from view 이름 ~ where 조건
- spark sql 사용
    - spark.sql('sql 쿼리문') # 지연연산

In [12]:
spark.sql('''
    select * from stocks
''') # 지연 연산

spark.sql('''
    select * from stocks
''').show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



4. stocks view에서 country가 Korea인 data만 추출하시오
- 조건이 있는 문제
- where 문을 사용한다.

In [14]:
spark.sql(''' 
    select * from stocks where country = 'Korea'
''') # 지연 연산

spark.sql(''' 
    select * from stocks where country = 'Korea'
''').show()

+-------+------+-------+------+--------+
|   name|ticker|country| price|currency|
+-------+------+-------+------+--------+
|Samsung|005930|  Korea| 70600|     KRW|
|  Kakao|035720|  Korea|125000|     KRW|
+-------+------+-------+------+--------+



5. df에서 country가 Korea인 data를 추출하시오
- df.method 의 조건 관련 함수는 filter, where
- filter(col('컬럼명'))/ df.컬럼명/ 조건의 문자열)

In [13]:
df.select('*')\
    .filter(df.country == 'Korea').show()

+-------+------+-------+------+--------+
|   name|ticker|country| price|currency|
+-------+------+-------+------+--------+
|Samsung|005930|  Korea| 70600|     KRW|
|  Kakao|035720|  Korea|125000|     KRW|
+-------+------+-------+------+--------+



6. stocks view에서 price 갸 2000 초과하고 country가  USA인 데이터의 `name`과 `price`를 출력하시오

In [17]:
spark.sql(''' 
    select name, price from stocks 
    where (country = 'USA') and (price > 2000)
''').show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
+------+-----+



In [18]:
# 7. df에서 price 가 2000 초과하고 country가  USA인 데이터의 name과 price를 출력하시오
df.select('name', 'price')\
    .filter((df.country == 'USA') & (df.price > 2000)).show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
+------+-----+



In [19]:
# 8. sotocks view에서 country가 U로 시작하는 데이터의 name과 price를 추출하시오

In [21]:
spark.sql('select name, price from stocks where country like "U%"').show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [22]:
# 9. df에서 country가 U로 시작하는 데이터의 name과 price를 추출하시오

In [27]:
df.select('name', 'price')\
    .filter(df.country.startswith('U')).show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [24]:
# 10. stocks view에서 countr가 'U'로 시작하고 이름에 'e'가 포함되지 않은 데이터의 name과 price를 추출하시오.

In [25]:
spark.sql('select name, price from stocks where country like "U%" and name not like "%e%"').show()

+------+-----+
|  name|price|
+------+-----+
|Amazon| 3518|
+------+-----+



11. df에서 countr가 'U'로 시작하고 이름에 'e'가 포함되지 않은 데이터의 name과 price를 추출하시오.
- df 의 not 연산자 : ~

In [26]:
df.select('name', 'price')\
    .filter((df.country.startswith('U')) & (~df.country.like('%e%'))).show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



12. stocks view에서 price가 1000~10000인 데이터를 추출하시오
- between min and max
- min > x and x < max 

In [30]:
spark.sql('select * from stocks where price between 1000 and 10000').show()

+------+------+-------+-----+--------+
|  name|ticker|country|price|currency|
+------+------+-------+-----+--------+
|Google| GOOGL|    USA| 2984|     USD|
|Amazon|  AMZN|    USA| 3518|     USD|
| Tesla|  TSLA|    USA| 1222|     USD|
|Toyota|  7203|  Japan| 2006|     JPY|
+------+------+-------+-----+--------+

